In [4]:
# !pip install -r requirements.txt

In [70]:
# !pip install torch==1.10.2+cpu torchvision==0.11.3+cpu torchaudio==0.10.2+cpu -f https://download.pytorch.org/whl/cpu/torch_stable.html

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
from datasets.transforms import ReidTransforms

In [4]:
# class ImageFolderWithPaths(ImageFolder):
#     """Custom dataset that includes image file paths. Extends
#     torchvision.datasets.ImageFolder
#     """

#     # override the __getitem__ method. this is the method that dataloader calls
#     def __getitem__(self, index):
#         # this is what ImageFolder normally returns 
#         original_tuple = super(ImageFolderWithPaths, self).__getitem__(index)
#         # the image file path
#         path = self.imgs[index][0]
#         # make a new tuple that includes original and the path
#         tuple_with_path = (original_tuple + (path,))
#         return tuple_with_path

def make_inference_data_loader(cfg, path, dataset_class):
    transforms_base = ReidTransforms(cfg)
    val_transforms = transforms_base.build_transforms(is_train=False)
    num_workers = cfg.DATALOADER.NUM_WORKERS
    val_set = dataset_class(path, val_transforms)
    val_loader = DataLoader(
        val_set, batch_size=cfg.TEST.IMS_PER_BATCH, shuffle=False, num_workers=num_workers
    )
    return val_loader


In [5]:
from PIL import Image
from torch.utils.data import Dataset

IMG_EXTENSIONS = (".jpg", ".jpeg", ".png", ".ppm", ".bmp", ".pgm", ".tif", ".tiff", ".webp")

from torchvision.datasets.folder import is_image_file

def pil_loader(path: str) -> Image.Image:
    # open path as file to avoid ResourceWarning (https://github.com/python-pillow/Pillow/issues/835)
    with open(path, "rb") as f:
        img = Image.open(f)
        return img.convert("RGB")

    
class ImageDataset(Dataset):
    """Image Person ReID Dataset"""

    def __init__(self, dataset: list, transform = None, loader=pil_loader):
        self.dataset = dataset
        self.transform = transform
        self.loader = loader

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, index):
        img_path = self.dataset[index]
        img = self.loader(img_path)

        if self.transform is not None:
            img = self.transform(img)

        return img, '', img_path ## Hack to be consistent with ImageFolderWithPaths dataset


In [6]:
from typing import List, Callable, Dict, Union
def get_all_images(path: Union[str, List[str]]) -> List[str]:
    if os.path.isdir(path):
        images = os.listdir(path)
        return [os.path.join(path, item) for item in images if is_image_file(item)]
    elif is_image_file(path):
        return [path]
    else:
        raise Exception(f"{path} is neither a path to a valid image file nor a path to folder containing images")

In [7]:
from config import cfg
import os

In [8]:
class args():
    pass
args.config_file = "configs/320_resnet50_ibn_a.yml"
args.opts = ["TEST.ONLY_TEST", "True", "GPU_IDS", []]

In [9]:
if args.config_file != "":
    cfg.merge_from_file(args.config_file)
cfg.merge_from_list(args.opts)

In [10]:
# test_path = 'test-images/gallery' # To test image taken from a single folder
# test_path = 'test-images-folder'  # To test image where each pids is stored in a seperate directory
# query_path = 'query.jpg'
# query_path = 'rolex_query.jpg'
query_path = 'test-query-images'

In [11]:
images_paths

NameError: name 'images_paths' is not defined

In [12]:
images_paths = get_all_images(query_path)

In [13]:
# val_loader = make_inference_data_loader(cfg, test_path, ImageFolderWithPaths)
val_loader = make_inference_data_loader(cfg, images_paths, ImageDataset)

In [14]:
for batch in val_loader:
#     print(batch[0], batch[1])
    print(batch[1], batch[2])

('', '') ('test-query-images/query.jpg', 'test-query-images/rolex_query.jpg')


In [15]:
from train_ctl_model import CTLModel
import pytorch_lightning as pl
import torch

In [16]:
# checkpoint_path = 'reid_weights.pth' ### This is just Pytorch wegihts therefore it requires different handling
# checkpoint_path = '/home/mwieczorek/centroids-reid/logs/df1_new/320_resnet50_ibn_a/train_ctl_model/version_2/auto_checkpoints/checkpoint_119.pth'
checkpoint_path = './auto_checkpoints/checkpoint_119.pth'

In [17]:
# or call with pretrained model
# model = CTLModel.load_from_checkpoint(checkpoint_path, map_location=torch.device('cpu'))
# model = CTLModel.load_from_checkpoint(checkpoint_path, map_location={'cuda': 'cpu'})
model = CTLModel(cfg, num_classes=1)
model.load_state_dict(state_dict=torch.load(checkpoint_path, map_location=torch.device('cpu'))['state_dict'], strict=False)

_IncompatibleKeys(missing_keys=[], unexpected_keys=['center_loss.centers', 'fc_query.weight'])

In [18]:
# model = model.cuda()

In [19]:
def _inference(model, batch, normalize_with_bn=True):
    model.eval()
    with torch.no_grad():
        data, _, filename = batch
        _, global_feat = model.backbone(data.cuda() if torch.cuda.is_available() else data)
        if normalize_with_bn:
            global_feat = model.bn(global_feat)
        return global_feat, filename

In [20]:
import numpy as np

In [21]:
vecs = []
paths = []

for pos, x in enumerate(val_loader):
#     if pos % print_freq == 0:
#         log.info(f'Number of processed images: {pos*cfg.TEST.IMS_PER_BATCH}')
    vec, path = _inference(model, x)
    for vv, pp in zip(vec, path):
        paths.append(pp)
        vecs.append(vv.detach().cpu().numpy())

all_vecs = np.vstack(vecs)
paths = np.array(paths)
all_vecs = np.array(all_vecs)

#     return all_vecs, paths

In [22]:
paths

array(['test-query-images/query.jpg', 'test-query-images/rolex_query.jpg'],
      dtype='<U33')

In [23]:
all_vecs.shape

(2, 2048)

### Load embeddings 

In [24]:
from pathlib import Path
LOAD_PATH = Path('gallery-data')

In [25]:
embeddings_gallery = torch.from_numpy(np.load(LOAD_PATH / 'embeddings.npy', allow_pickle=True))
paths_gallery = np.load(LOAD_PATH / 'paths.npy', allow_pickle=True)

In [26]:
normalize_features = True
if normalize_features:
    embeddings_gallery = torch.nn.functional.normalize(embeddings_gallery, dim=1, p=2)
    all_vecs = torch.nn.functional.normalize(torch.from_numpy(all_vecs), dim=1, p=2)
else:
    all_vecs = torch.from_numpy(all_vecs)

In [27]:
use_cuda = False
device = torch.device('cuda') if use_cuda else torch.device('cpu')
embeddings_gallery = embeddings_gallery.to(device)
all_vecs = all_vecs.to(device)

### Calculate similarity

In [28]:
from utils.reid_metric import get_dist_func

In [29]:
# dist_func = get_dist_func(cfg.SOLVER.DISTANCE_FUNC)
dist_func = get_dist_func('cosine')

Using cosine as distance function during evaluation


In [30]:
distmat = dist_func(x=all_vecs, y=embeddings_gallery).cpu().numpy()
indices = np.argsort(distmat, axis=1)

In [31]:
distmat

array([[1.0135438 , 0.9371855 , 0.9554886 , 0.9233496 , 0.9608762 ,
        0.8070517 , 0.89708567, 0.75736654, 0.8523031 , 0.7931752 ],
       [0.8596092 , 0.75821984, 1.0694846 , 1.1070676 , 1.0446938 ,
        1.0283605 , 0.95984876, 1.032609  , 0.9873477 , 0.9749332 ]],
      dtype=float32)

In [32]:
indices

array([[7, 9, 5, 8, 6, 3, 1, 2, 4, 0],
       [1, 0, 6, 9, 8, 5, 7, 4, 2, 3]])

In [33]:
type(all_vecs)

torch.Tensor

In [34]:
topk = 3

In [35]:
paths_gallery[indices][:, :topk]

array([['test-images-folder/wc2120102spa-027/wc2120102spa-027_06.jpg',
        'test-images-folder/wc2121101spa-021/wc2121101spa-021_06.jpg',
        'test-images-folder/wc2120102spa-027/wc2120102spa-027_04.jpg'],
       ['test-images-folder/rolex/rolex_gallery.jpg',
        'test-images-folder/apple_watch/apple_watch_gallery.jpg',
        'test-images-folder/wc2120102spa-027/wc2120102spa-027_05.jpg']],
      dtype='<U59')

In [36]:
paths

array(['test-query-images/query.jpg', 'test-query-images/rolex_query.jpg'],
      dtype='<U33')

In [39]:
indices_top = indices[:, :topk]

In [40]:
out = {
    query_path: {
        "indices": indices_top[q_num, :],
        "paths": paths_gallery[indices_top[q_num, :]],
        "distances": distmat[q_num, indices_top[q_num, :]]
    }
    for q_num, query_path in enumerate(paths)
}

In [42]:
out

{'test-query-images/query.jpg': {'indices': array([7, 9, 5]),
  'paths': array(['test-images-folder/wc2120102spa-027/wc2120102spa-027_06.jpg',
         'test-images-folder/wc2121101spa-021/wc2121101spa-021_06.jpg',
         'test-images-folder/wc2120102spa-027/wc2120102spa-027_04.jpg'],
        dtype='<U59'),
  'distances': array([0.75736654, 0.7931752 , 0.8070517 ], dtype=float32)},
 'test-query-images/rolex_query.jpg': {'indices': array([1, 0, 6]),
  'paths': array(['test-images-folder/rolex/rolex_gallery.jpg',
         'test-images-folder/apple_watch/apple_watch_gallery.jpg',
         'test-images-folder/wc2120102spa-027/wc2120102spa-027_05.jpg'],
        dtype='<U59'),
  'distances': array([0.75821984, 0.8596092 , 0.95984876], dtype=float32)}}

In [37]:
out = {
    query_path: {
        "indices": indices[q_num, :],
        "paths": paths_gallery[indices[q_num, :]],
        "distances": distmat[q_num, indices[q_num, :]]
    }
    for q_num, query_path in enumerate(paths)
}

In [38]:
out

{'test-query-images/query.jpg': {'indices': array([7, 9, 5, 8, 6, 3, 1, 2, 4, 0]),
  'paths': array(['test-images-folder/wc2120102spa-027/wc2120102spa-027_06.jpg',
         'test-images-folder/wc2121101spa-021/wc2121101spa-021_06.jpg',
         'test-images-folder/wc2120102spa-027/wc2120102spa-027_04.jpg',
         'test-images-folder/wc2121101spa-021/wc2121101spa-021_04.jpg',
         'test-images-folder/wc2120102spa-027/wc2120102spa-027_05.jpg',
         'test-images-folder/wc2120102evb-011/wc2120102evb-011_05.jpg',
         'test-images-folder/rolex/rolex_gallery.jpg',
         'test-images-folder/wc2120102evb-011/wc2120102evb-011_04.jpg',
         'test-images-folder/wc2120102evb-011/wc2120102evb-011_07.jpg',
         'test-images-folder/apple_watch/apple_watch_gallery.jpg'],
        dtype='<U59'),
  'distances': array([0.75736654, 0.7931752 , 0.8070517 , 0.8523031 , 0.89708567,
         0.9233496 , 0.9371855 , 0.9554886 , 0.9608762 , 1.0135438 ],
        dtype=float32)},
 'test-qu

In [232]:
SAVE_PATH = Path('./results.npy')
np.save(SAVE_PATH, out)

In [233]:
out

{'test-query-images/query.jpg': {'indices': array([7, 9, 5, 8, 6, 3, 1, 2, 4, 0]),
  'paths': array(['test-images-folder/wc2120102spa-027/wc2120102spa-027_06.jpg',
         'test-images-folder/wc2121101spa-021/wc2121101spa-021_06.jpg',
         'test-images-folder/wc2120102spa-027/wc2120102spa-027_04.jpg',
         'test-images-folder/wc2121101spa-021/wc2121101spa-021_04.jpg',
         'test-images-folder/wc2120102spa-027/wc2120102spa-027_05.jpg',
         'test-images-folder/wc2120102evb-011/wc2120102evb-011_05.jpg',
         'test-images-folder/rolex/rolex_gallery.jpg',
         'test-images-folder/wc2120102evb-011/wc2120102evb-011_04.jpg',
         'test-images-folder/wc2120102evb-011/wc2120102evb-011_07.jpg',
         'test-images-folder/apple_watch/apple_watch_gallery.jpg'],
        dtype='<U59'),
  'distances': array([0.75736654, 0.7931752 , 0.8070517 , 0.8523031 , 0.89708567,
         0.9233496 , 0.9371855 , 0.9554886 , 0.9608762 , 1.0135438 ],
        dtype=float32)},
 'test-qu